# OptalCP Python API Examples

This notebook demonstrates the OptalCP Python API.

In [1]:
import optalcp as cp
import asyncio
import os

# Temporary: set the solver path via environment variable
os.environ["OPTALCP_SOLVER"] = "/home/lukas/optacp/lib/python3.12/site-packages/optalcp_bin_academic/bin/linux-x64/optalcp"
print(f"OPTALCP_SOLVER set to: {os.environ['OPTALCP_SOLVER']}")
print(f"OptalCP version: {cp.__version__}")

OPTALCP_SOLVER set to: /home/lukas/optacp/lib/python3.12/site-packages/optalcp_bin_academic/bin/linux-x64/optalcp
OptalCP version: 2025.11.1


## 1. Basic Modeling

This example shows how to create interval variables, add constraints, and minimize an objective.

In [2]:
import optalcp as cp
model = cp.Model()

# Create two tasks
x = model.interval_var(length=10, name="x", optional=True)
y = model.interval_var(length=20, name="y")

# Fluent API: call start() and end() directly on interval variables
model.constraint(x.start() >= 0)
model.constraint(y.end() <= 100)

# Fluent API: end_before_start() called directly on x (instead of model.end_before_start(x, y))
x.end_before_start(y)
# Equivalent to:
model.constraint(x.end() <= y.start())

# Minimize completion time
model.minimize(y.end())

result = cp.solve(model)
print(f"Objective: {result.objective_value}")

# Without the following line, typing will warn that best_solution may be None
assert(result.best_solution)
# In the best solution x is absent. It's start time is None in this case:
print(f"x starts at: {result.best_solution.get_start(x)}")

--------------------------------------------------------------------------------
                         ScheduleOpt OptalCP [Academic]
                           Version 2025.11.1 (Linux)
                CPU: AMD Ryzen 5 5500U with (12 physical cores)
--------------------------------------------------------------------------------
Input parse time: 00:00
Input:
   0 integer variables, 2 interval variables, 4 constraints, 10kB
   00:00 Presolving..
Presolved:
   0 integer variables, 2 interval variables, 2 constraints, 9.6kB
   00:00 Starting the search using 12 workers (physical cores count).
--------------------------------------------------------------------------------
   00:00 Lower bound 20 Worker 7
   00:00 Solution 30 Worker 7: LNS
   00:00 Solution 20 Worker 7: LNS
   00:00 Worker 7: The current best solution is optimal.
--------------------------------------------------------------------------------
   Objective value: 20 (optimal)
       Lower bound: 20
         Solutions: 

## 2. no_overlap Constraint

The `no_overlap` constraint ensures tasks don't overlap in time.

In [3]:
# Basic no_overlap: tasks cannot overlap in time
model = cp.Model()

tasks = [
    model.interval_var(length=10, name="Task1"),
    model.interval_var(length=20, name="Task2"),
    model.interval_var(length=15, name="Task3")
]
transitions = [
    [0, 5, 10],
    [5, 0, 5],
    [10, 5, 0]
]

model.constraint(model.no_overlap(tasks, transitions))

# Fluent API: t.end() called directly on each task in generator expression
model.minimize(model.max(t.end() for t in tasks))

# Solve and check
result = cp.solve(model, cp.Parameters(logLevel=0))
print(f"Found solution: {result.nb_solutions > 0}")
print(f"Duration: {result.duration:.3f}s")

Found solution: True
Duration: 0.063s


## 3. Cumulative Constraints

In [4]:
model = cp.Model()

task1 = model.interval_var(length=10, name="task1")
task2 = model.interval_var(length=15, name="task2")
task3 = model.interval_var(length=20, name="task3", optional=True)

# Usage of task1 is a variable:
usage1 = model.int_var(min=3, max=5, name="usage1", optional=True)

# Total resource usage must not exceed capacity of 8
model.constraint(
        model.pulse(task1, usage1) +
        model.pulse(task2, 2) +
        model.pulse(task3, 4) <= 8
)

# Synchronize presence of usage1 and task1
model.constraint(usage1.presence() == task1.presence())

result = cp.solve(model, params=cp.Parameters(solutionLimit=2, logLevel=0))
print(f"Found {result.nb_solutions} solutions.")

Found 2 solutions.


## 4. Python trickery

Using `sum` to combine multiple pulses.

In [5]:
model = cp.Model()

tasks = [model.interval_var(length=10, name=f"task_{i}") for i in range(5)]
demands = [3, 2, 4, 1, 3]

cumul = model.sum(task.pulse(demand) for task, demand in zip(tasks, demands))

# Operators are overloaded on both sides (<= and >=):
model.constraint(10 >= cumul)

result = cp.solve(model, params=cp.Parameters(solutionLimit = 1, logLevel=0))
print(f"Found solution: {result.nb_solutions > 0}")
print(f"Duration: {result.duration:.3f}s")

Found solution: True
Duration: 0.020s


## 5. Step Functions

Step functions represent piecewise constant functions over time, useful for modeling calendars and time-based costs.

In [6]:
model = cp.Model()

# Create a calendar where 0=forbidden (weekend), 1=allowed (weekday)
# Week 1: Mon-Fri (days 0-4), Sat-Sun (days 5-6)
# Week 2: Mon-Fri (days 7-11)
calendar = model.step_function([
    (0, 1),    # Allowed from Monday (day 0)
    (5, 0),    # Forbidden from Saturday (day 5)
    (7, 1),    # Allowed from next Monday (day 7)
    (12, 0)    # Forbidden from next Saturday (day 12)
])

# Create a task. First start time is Saturday (day 5)
task = model.interval_var(length=3, name="task", start=(5, 20))

# Fluent API: task.forbid_start(calendar) instead of model.forbid_start(task, calendar)
task.forbid_start(calendar)
model.minimize(task.end())

result = cp.solve(model, cp.Parameters(logLevel=0))
print(f"Objective: {result.objective_value}")
assert(result.best_solution)
# The first feasible start time is Monday (day 7):
print(f"Task starts at: {result.best_solution.get_start(task)}")

Objective: 10.0
Task starts at: 7


## 6. Set worker parameters

In [7]:
model = cp.Model()
x = model.interval_var(length=10, name="x")
model.minimize(x.start())

# Parameter class:
params = cp.Parameters()
params.timeLimit = 60
params.solutionLimit = 1
params.searchType = "LNS"
params.nbWorkers = 3

fdsWorker = cp.WorkerParameters(searchType="FDS")

# Parameters for individual workers:
params.workers = [
    cp.WorkerParameters(searchType="FDSLB"),
    fdsWorker,
    fdsWorker
]

result = cp.solve(model, params=params)

--------------------------------------------------------------------------------
                         ScheduleOpt OptalCP [Academic]
                           Version 2025.11.1 (Linux)
                CPU: AMD Ryzen 5 5500U with (12 physical cores)
--------------------------------------------------------------------------------
Input parse time: 00:00
Parameters:
   NbWorkers = 3
   TimeLimit = 60 seconds
   SolutionLimit = 1
   Worker 0: SearchType = FDSLB
   Worker 1: SearchType = FDS
   Worker 2: SearchType = FDS
Input:
   0 integer variables, 1 interval variables, 0 constraints, 9.15kB
   00:00 Presolving..
Presolved:
   0 integer variables, 1 interval variables, 0 constraints, 9.14kB
   00:00 Starting the search using 3 workers (nbWorkers parameter).
--------------------------------------------------------------------------------
   00:00 Lower bound 0 Worker 1
   00:00 Solution 0 Worker 0: SimpleLB
   00:00 Solution limit reached (1).
----------------------------------------

## 7. Async Solver with Event Handlers

The `Solver` class provides async support and event handlers for monitoring the solve process.

In [8]:
# Create solver
solver = cp.Solver()
solver.output_stream = None  # Disable default output

# Set event handlers via properties
solver.on_log = lambda msg: print(f"LOG: {msg}", end='')
solver.on_solution = lambda e: print(f"SOLUTION AT {e['solveTime']:.2f}s, objective={e.get('objective', 'N/A')}")
solver.on_lower_bound = lambda e: print(f"LB: {e['value']}")

# Build model
model = cp.Model()
x = model.interval_var(length=10, name="x")
y = model.interval_var(length=20, name="y")

# Fluent API: x.end_before_start(y) and y.end() called directly on interval variables
x.end_before_start(y)
model.minimize(y.end())

# Solve asynchronously
result = await solver.solve(model)

print(f"\nFinal result: {result.nb_solutions} solutions, objective={result.objective_value}")

LOG: --------------------------------------------------------------------------------
                         ScheduleOpt OptalCP [Academic]
                           Version 2025.11.1 (Linux)
                CPU: AMD Ryzen 5 5500U with (12 physical cores)
--------------------------------------------------------------------------------
LOG: Input parse time: 00:00
LOG: Input:
   0 integer variables, 2 interval variables, 1 constraints, 9.38kB
LOG:    00:00 Presolving..
LOG: Presolved:
   0 integer variables, 2 interval variables, 1 constraints, 9.37kB
LOG:    00:00 Starting the search using 12 workers (physical cores count).
--------------------------------------------------------------------------------
LOG:    00:00 Lower bound 30 Worker 2
LB: 30.0
SOLUTION AT 0.01s, objective=N/A
LOG:    00:00 Solution 30 Worker 2: LNS
LOG:    00:00 Worker 2: The current best solution is optimal.
LOG: --------------------------------------------------------------------------------
   Objective val